下面是找出来的几个候选  
pdfminer3k  
pdfminer.six  
pypdf  
pypdfocr 


In [1]:
# file paths
phone_numbers = ["./data/phone_numbers/070219_KV_BR_Telefonverzei.pdf",
                "./data/phone_numbers/52_pdf_wegweiser_schlaganfall.pdf",
                 "./data/phone_numbers/FL_SYB_BetriebsaerztlicherDienst_ID8414.pdf",
                 "./data/phone_numbers/Infoblatt_154.pdf"
                ]
scans = ["./data/scans/double_ocr.pdf",
         "./data/scans/single_ocr.pdf"]

### pdfminer.six
```
pip install pdfminer.six
```

In [7]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io

def convert(case,fname, pages=None):
    if not pages: pagenums = set();
    else:         pagenums = set(pages);      
    manager = PDFResourceManager() 
    codec = 'utf-8'
    caching = True

    if case == 'text' :
        output = io.StringIO()
        converter = TextConverter(manager, output, codec=codec, laparams=LAParams())     
    if case == 'HTML' :
        output = io.BytesIO()
        converter = HTMLConverter(manager, output, codec=codec, laparams=LAParams())

    interpreter = PDFPageInterpreter(manager, converter)   
    infile = open(fname, 'rb')

    for page in PDFPage.get_pages(infile, pagenums,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    convertedPDF = output.getvalue()  

    infile.close(); converter.close(); output.close()
    return convertedPDF

#//////////// main ///////////////////////
filePDF  = phone_numbers[0]     # input
fileHTML = 'myHTML.html'   # output
fileTXT  = 'myTXT.txt'     # output

case = "text"

if case == 'HTML' :
    convertedPDF = convert('HTML', filePDF)
    fileConverted = open(fileHTML, "wb")
if case == 'text' :
    convertedPDF = convert('text', filePDF)
    fileConverted = open(fileTXT, "w", encoding="utf-8")

print(convertedPDF)
fileConverted.write(convertedPDF)
fileConverted.close()
#print(convertedPDF) 

UniversitätsKlinikum Heidelberg

TelefonBuch
So können Sie uns erreichen!
Klinik für Kinder- und Jugendmedizin
· Notfallambulanzen
· Ambulanzen
· Stationäre Aufnahme

Stand Januar 2007
www.klinikum.uni-heidelberg.de

Abteilung Kinderheilkunde I
Schwerpunkt Allgemeine Pädiatrie · Stoffwechsel · 
XXXXXXXX

Inhaltsverzeichnis
Telefonbuch Universitätsklinikum Heidelberg

Telefonzentrale und Notfallambulanzen 

Anästhesiologische Klinik 

Augenklinik 

Chirurgische Klinik 
Klinik für Allgemein-, Viszeral- , Unfall-  und Kinderchirurgie
Klinik für Herzchirurgie
Klinik für Gefäßchirurgie
Klinik für Urologie
 
Frauenklinik 
Allgemeine Frauenheilkunde und Geburtshilfe
Gynäkologische Endokrinologie und Fertilitätsstörungen

Klinik für Hals-, Nasen-, Ohrenheilkunde 

Hautklinik 
 
Heidelberger Ionenstrahl-Therapiezentrum (HIT) 

  42

Klinik für Kinder- und Jugendmedizin 
Kinderheilkunde I
Allgemeine Pädiatrie, Stoffwechsel-Gastroenterologie-Nephrologie
Kinderheilkunde II 
Kinderkardiologie
Kind

### PDF TO Text
For Debian, Ubuntu, and friends
```
sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev
```
For Fedora, Red Hat, and friends
```
sudo yum install gcc-c++ pkgconfig poppler-cpp-devel python3-devel
```
For Mac OS
```
brew install pkg-config poppler
```
then run
```
pip install pdftotext
```

In [14]:
import pdftotext

'''
 # Load your PDF
with open(phone_numbers[1], "rb") as f:
    pdf = pdftotext.PDF(f)

 # Save all text to a txt file.
with open('test.txt', 'w') as f:
    f.write("\n\n".join(pdf))
'''

def get_text_pdftotext(pdf_files):
    for file in pdf_files:
        with open(file, "rb") as f:
            pdf = pdftotext.PDF(f)
            for page in pdf:
                print(page)

get_text_pdftotext([scans[1]])

                                                    Altlußheim                                          (0.62.05)                                                                     |                                                                            ne.                                                                   www.dastelefonbuch.de
                                                                                                                     Gottfried Werner Tulla-20 ........020...0.... 32030 | Heid Dagmar Rosemarie.................. 364 1733 | Hoffstätter Dieter u. Fischer Beate......... 31973
                          AESE5-APOTHEKE|                                                                            cottmam Eisabeth Hause                                                        31942 | Erich-Kästner-Weg 1A                                                                                                   Kurpfalz-9
                                           